In [1]:
import os
os.chdir('..')

from util import CONFIG
CONFIG.set_use_mpl_tables(True)

In [2]:
from math import log2
import numpy as np

from sim_circuit import QuantumRegister, QuantumCircuit


def phase_estimation_unitary(n, U, swap=True):
    assert(U.shape[0] == U.shape[1])
    m = int(log2(U.shape[0]))

    eigvals, eigvecs = np.linalg.eig(U)

    q = QuantumRegister(n)
    a = QuantumRegister(m)
    qc = QuantumCircuit(q, a) # ancilla is last

    qc.append_u(eigvecs, a)
    qc.report('eigenstate')

    for i in range(n):
        qc.h(q[i])

    for i in range(n):
        if swap:
            for _ in range(2**i):
                qc.c_append_u(U, q[i], a)
        else:
            # qubit reversal
            for _ in range(2**i):
                qc.c_append_u(U, q[n-1-i], a)
            # decreasing powers of 2
            # for _ in range(2**(n-1-i)):
            #     qc.c_append_u(U, q[i], a)

    qc.report('geometric_sequence_superposition')

    qc.append_u(np.conj(eigvecs.transpose()), a)

    qc.report('geometric_sequence')

    qc.iqft(q if swap else q[::-1], swap)
    qc.report('estimate')

    return qc

In [3]:
    import scipy.stats
    
    n=3
    m = 2
    
    U = scipy.stats.unitary_group.rvs(2**m)
    qc = phase_estimation_unitary(n, U, swap=True)

In [4]:
from util import print_state_table

print_state_table(qc.run())


Outcome  Binary  Amplitude           Magnitude  Direction  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        00000   0.0333 - i0.0372    0.0499      -48.83°   █                         0.0025
1        00001   0.0572 - i0.0275    0.0635      -25.32°   █                         0.004 
2        00010   0.1099 - i0.0061    0.1101       -3.82°   ██                        0.0121
3        00011   0.9209 + i0.3228    0.9758       19.32°   ███████████████████████   0.9522
4        00100  -0.1037 - i0.0927    0.1391     -138.79°   ███                       0.0193
5        00101  -0.0303 - i0.0630    0.0699     -115.31°   █                         0.0049
6        00110  -0.0029 - i0.0519    0.0519      -93.80°   █                         0.0027
7        00111   0.0156 - i0.0444    0.047       -70.36°   █                         0.0022
8        01000   0.0000 + i0.0000    0.0                             